<a href="https://colab.research.google.com/github/UmatmU/UmatmU.GitHub.io/blob/gh-pages/Thesis_project_halo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Run this command twice
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install --no-pin pyccl
import pyccl as ccl

✨🍰✨ Everything looks OK!
Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.1.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [1]:
import numpy as np
import pylab as plt
import matplotlib.cm as cm
from scipy import integrate
from scipy import interpolate
from functools import partial
%matplotlib inline

ModuleNotFoundError: No module named 'pyccl'

In [ ]:
# Cosmology
cosmo = ccl.Cosmology(Omega_c=0.196, Omega_b=0.042, h=0.73, sigma8=0.74, n_s=0.951)
# Omega_c=0.2265, Omega_b=0.0455, h=0.704, sigma8=0.81 , n_s=0.967) # WMAP7 cosmology from vanDaalen 2019

# size of k
kmin = 1E-2; kmax = 1
# Wavenumbers
k_arr = np.geomspace(kmin,kmax,256)  # np.geomspace() similar to np.logspace(), but with endpoints specified directly; e.g. np.geomspace(1, 256, num=4), output: array([(256)^(0/3), (256)^(1/3), (256)^(2/3), 256]). remarks: for pt. start from non-one # e.g. np.geomspace(1E-2,1,4), just shift the starting pt to one, output: [0.01*0.01^(-0/3), 0.01*0.01^(-1/3), 0.01*0.01^(-2/3) , 0.01*0.01^(-3/3)]

# Scale factor
a_sf=1

# We will use a mass definition with Delta = 200 times the matter density
hmd_200m = ccl.halos.MassDef(200, 'matter')
# The Tinker 2008 mass function
hmf_200m = ccl.halos.MassFuncTinker08(mass_def=hmd_200m, mass_def_strict=False)
# The Tinker 2010 halo bias
hbf = ccl.halos.HaloBiasTinker10(mass_def=hmd_200m, mass_def_strict=False)
# The Duffy 2008 concentration-mass relation
cM = ccl.halos.ConcentrationDuffy08(mass_def=hmd_200m)
# The NFW profile to characterize the matter density around halos
pM = ccl.halos.HaloProfileNFW(mass_def=hmd_200m, concentration=cM)
hmc = ccl.halos.HMCalculator(mass_function=hmf_200m, halo_bias=hbf,mass_def=hmd_200m)
print(hmd_200m)
# Define some useful densities
rho_m = cosmo['Omega_m']*ccl.physical_constants.RHO_CRITICAL*cosmo['h']**2
rho_DM = cosmo['Omega_c']*ccl.physical_constants.RHO_CRITICAL*cosmo['h']**2
rho_g = rho_m - rho_DM

#For the gas mass fraction within halos
m_0g = 1E12/cosmo['h']
sigmag = 3

#For the gas profile
beta = 2.9

#Parameters that inform the power spectrum of the gas,
#specifically the extended component.
Fg = 0.05 #this is an example, it should be <<1
bd = 0.85 #from Fedeli

# A reference halo mass that will be used later
mhalo = 1E14/cosmo['h']

# Array of masses
mmin_DM=1E6
mmax_DM=1E16
mmin_gas=m_0g
mmax_gas=mmax_DM
m_arr = np.geomspace(mmin_DM,mmax_DM,128)

Plot the f_gas vs M for Mead's halo model.

In [ ]:
def f_gas(m):
    fgas = []
    for i in m:
        if i<m_0g:
            fgas.append(0.)
        else:
            fgas.append((cosmo['Omega_b']/cosmo['Omega_m'])*(i/m_0g)**beta/(1+(i/m_0g)**beta)) # fitting parameter beta>0
    return fgas

plt.figure()
plt.plot(m_arr*cosmo['h'],f_gas(m_arr), 'b-')
plt.xscale('log')
plt.xlabel(r'$M$ $[$M$_\odot/h]$',fontsize=18)
plt.ylabel(r'$f_{\rm gas}(M)$',fontsize=18)
plt.title('Plot of ' r'$f_{\rm gas}(M)$ versus M',fontsize=18)
plt.show()

Plot the P_b/P_dm vs k figure for Mead's halo model.

In [ ]:
#Interpolate the profiles for speed
row = 150; col = 2000
M = np.logspace(np.log10(mmin_DM),np.log10(mmax_DM),num=row)  # list of DM masses spreaded by log-spacing; np.logspace(start,stop,num) e.g. np.logspace(2.0,3.0,num=4), output: array([10^2, 10^(2+1/3), 10^(2+2/3), 10^3])
yDM=np.zeros((row,col))
M2D = np.zeros((row,col))  # store the list of masses repeatedly in multiple columns
for i in range(0,col):
    M2D[:,i]=M
K = np.linspace(kmin,kmax,num=col)  # e.g. np.linspace(2.0, 3.0, num=5), output: array([2., 2.25, 2.5, 2.75, 3.])
for j in range(0,row):
    yDM[j,:]=pM.fourier(cosmo, K, M[j], a_sf)/M[j]
fDM = interpolate.interp2d(M,K,np.transpose(yDM))
print("DM profiles interpolated...\n")

# Halo Mass Calculator (for DM)
hmc = ccl.halos.HMCalculator(mass_function=hmf_200m, halo_bias=hbf, mass_def=hmd_200m)

# Generic integrals needed
# From Mead (2020)
def integrandDM(m,k,prof1):
    dndlog10m = hmf_200m(cosmo, m, a_sf)
    dndm=dndlog10m/(m*np.log(10.))
    bm = hbf(cosmo,m,a_sf)
    W = prof1(m,k)
    return dndm*bm*W*m
def integratedDM(mmin,mmax,k,p1):
    return integrate.quad(integrandDM, mmin, mmax, args=(k,p1),epsabs=0,epsrel=1E-4,limit=5000)[0]

# def integrandGENbp(m,k,prof1):
#     bm = hbf(cosmo,m,a_sf)
#     dndlog10m = hmf_200m(cosmo, m, a_sf)
#     dndm=dndlog10m/(m*np.log(10.))
#     y=prof1(m,k)
#     return dndm*m*bm*y
# def integratedGENbp(mmin,mmax,k,p1):
#     return integrate.quad(integrandGENbp, mmin, mmax, args=(k,p1),epsabs=0,epsrel=1E-3,limit=5000)[0]

def f_g(m):
    if m<m_0g:
      fg=0
    else:
      fg=(cosmo['Omega_b']/cosmo['Omega_m'])*(m/m_0g)**beta/(1+(m/m_0g)**beta) # fitting parameter beta>0
    return fg

def integrandB(m,k,prof1):
    dndlog10m = hmf_200m(cosmo, m, a_sf)
    dndm=dndlog10m/(m*np.log(10.))
    bm = hbf(cosmo,m,a_sf)
    Wb = prof1(m,k)*(f_g(m))
    return dndm*bm*Wb*m

def integratedB(mmin,mmax,k,p1):
    return integrate.quad(integrandB, mmin, mmax, args=(k,p1),epsabs=0,epsrel=1E-4,limit=5000)[0]

# power spectrum, PDM2h for DM; PB2h for baryon
# def PDM2h(k):
#     return ccl.linear_matter_power(cosmo, k, a_sf)*integratedGENbp(mmin_DM,mmax_DM,k,fDM)**2

def PDM2h(k):
    return ccl.linear_matter_power(cosmo, k, a_sf)*integratedDM(mmin_DM,mmax_DM,k,fDM)**2

def PB2h(k):
    return ccl.linear_matter_power(cosmo, k, a_sf)*(integratedB(mmin_gas,mmax_DM,k,fDM)+cosmo['Omega_c']/cosmo['Omega_m']*integratedDM(mmin_DM,mmax_DM,k,fDM))**2

PDM2h_arr=np.zeros(len(k_arr))
PB2h_arr=np.zeros(len(k_arr))
for i in range(0,len(k_arr)):
    PDM2h_arr[i]=PDM2h(k_arr[i])
    PB2h_arr[i]=PB2h(k_arr[i])

Plot the P_b and P_dm vs k respectively for Mead's halo model.

In [ ]:
plt.figure(figsize=(7,6))

# Non-linear matter power spectrum, for comparison
pk_nl = ccl.nonlin_matter_power(cosmo, k_arr, a_sf)

# Plot for the ratio P_DM/P_nl and P_b/P_nl of 2-halo
plt.plot(k_arr/cosmo['h'], (1/rho_m)**2*PDM2h_arr/pk_nl, 'k-', label='$P_{DM}^{2H}(k)$')
plt.plot(k_arr/cosmo['h'], (1/rho_m)**2*PB2h_arr/pk_nl, 'r-', label='$P_{B}^{2H}(k)$')

print(PDM2h_arr[20])
print(pk_nl[20])

plt.xscale('log')
plt.yscale('log')
plt.legend(loc='lower left', fontsize=12.5)
plt.ylabel(r'$P^{2H}(k)/P_{nl}(k)$', fontsize=15)
plt.xlabel(r'$k\,\,[{\rm h/Mpc}]$', fontsize=15)
plt.title('Plot of $P^{2H}(k)/P_{nl}(k)$ versus k',fontsize=18)

plt.show()

Plot the P_b/P_dm vs k figure for Mead's halo model.

In [ ]:
from scipy.optimize import curve_fit

# The reduced ratio function
def f(k,n):
  return 1-n*k**2

plt.figure(figsize=(7,6))

# # Plot for the ratio P_b/P_DM of 2-halo
# plt.plot(k_arr/cosmo['h'], PB2h_arr/PDM2h_arr, 'k-')

# Fit the curve
plt.scatter(k_arr/cosmo['h'], PB2h_arr/PDM2h_arr, s=3, color='r')
popt, _ = curve_fit(f, k_arr/cosmo['h'], PB2h_arr/PDM2h_arr) # popt: optimal/coefficient value
plt.plot(k_arr/cosmo['h'], f(k_arr/cosmo['h'], *popt), color='black', label='Slope of ratio/k = %.3f' % tuple(popt))

plt.xscale('log')
plt.yscale('log')
plt.ylabel(r'$P_{B}^{2H}(k)/P_{DM}^{2H}(k)$', fontsize=15)
plt.xlabel(r'$k\,\,[{\rm h/Mpc}]$', fontsize=15)
plt.title('Plot of $P_{B}^{2H}(k)/P_{DM}^{2H}(k)$ versus k', fontsize=18)

plt.show()